En este notebook, se transforman los archivos de Yelp y Google a formato parquet.

In [1]:
from glob import glob
import warnings
import pandas as pd

In [3]:
# Opción para ignorar advertencias
warnings.filterwarnings('ignore')
# Opción que deshabilitaa el limite de columnas mostradas
pd.set_option('display.max_columns', None)

---
## Dataset Yelp

### `business.pkl`

In [ ]:
business_path = './data/Yelp/business.pkl'
# Leer pkl (pickle)
df = pd.read_pickle(business_path)
df.sample(2)

* Se observan columnas con nombres duplicados. Hay que unir estas columns.

In [ ]:
# Calcular la cantida de columns dividida por 2
punto_medio = len(df.columns) // 2
# Dividir el dataframe por mitad, a largo de las columns,
# utilizando el punto_medio calculado
df_1er_mitad = df.iloc[:, :punto_medio]
df_2nd_mitad = df.iloc[:, punto_medio:]
# Unir los dataframes resultantes
df_reorganizado = pd.concat([df_1er_mitad, df_2nd_mitad], axis=0)
df_reorganizado.sample(2)

In [ ]:
# Eliminar filas con todo nulos
df_reorganizado.dropna(axis=0, how='all', inplace=True)

In [ ]:
# Almacenar en formato parquet, en el mismo directorio de 'business_path'
df_reorganizado.to_parquet(business_path.replace('.pkl', '.parquet'))  # se cambia sufijo '.pkl' a '.parquet'

### `checkin.json`

In [ ]:
checkin_path = './data/Yelp/checkin.json'
# Leer JSON
df = pd.read_json(checkin_path, lines=True)
# Almacenar en formato parquet, en el mismo directorio
df.to_parquet(checkin_path.replace('.json', '.parquet'))  # se cambia sufijo '.json' a '.parquet'

### `review.json`

! Transformacion no posible en este ambiente

In [ ]:
#review_path = './data/Yelp/review.json'Merge de parquets
# Leer JSON
#df = pd.read_json(review_path, lines=True)
# Almacenar en formato parquet, en el mismo directorio
#df.to_parquet(review_path.replace('.json', '.parquet'))  # se cambia sufijo '.json' a '.parquet'

In [ ]:
#from pyarrow import json
#import pyarrow.parquet as pq

#review_path = './data/Yelp/review.json'
# Leer JSON como tabla parquet
#tabla = json.read_json(review_path)
# Almacenar en formato parquet, en el mismo directorio
#pq.write_table(tabla, review_path.replace('.json', '.parquet'))

### `tip.json`

In [ ]:
tip_path = './data/Yelp/tip.json'
# Leer JSON
df = pd.read_json(tip_path, lines=True)
# Almacenar en formato parquet, en el mismo directorio
df.to_parquet(tip_path.replace('.json', '.parquet'))  # se cambia sufijo '.json' a '.parquet'Merge de parquets

### `user.parquet`

Esta en formato parquet. Solo visualizamos.

In [ ]:
user_df = pd.read_parquet('./data/Yelp/user.parquet')
user_df.sample(2)

---
## Dataset Google

### `metadata-sitios/`

In [2]:
# Crear lista de JSON en /metadata-sitios/
google_dir = './data/Google'
metadata_jsons = glob(f'{google_dir}/metadata-sitios/*json')
metadata_jsons

[]

In [ ]:
## Convertir archivos JSON, en /metadata-sitios/, a parquet

# Crear dataframe donde se unen los datos extraidos de JSON
union_df = pd.DataFrame()
# Iterar por cada JSON dentro de /metadata-sitios/
for f_json in metadata_jsons:
    # Leer JSON
    df = pd.read_json(f_json, lines=True)
    # Unir a union_df
    union_df = pd.concat([union_df, df], ignore_index=True)

# Almacenar en formato parquet, en el mismo directorio
union_df.to_parquet(f'{google_dir}/metadata-sitios/metadata_sitios.parquet')

### `reviews-estados/`

In [3]:
# Lista de directorios en /reviews-estados/
google_dir = './data/Google'
review_dirs = glob(f'{google_dir}/reviews-estados/*')
review_dirs

['./data/Google/reviews-estados/review-Maryland',
 './data/Google/reviews-estados/review_Mississippi.parquet',
 './data/Google/reviews-estados/review-New_Hampshire',
 './data/Google/reviews-estados/review_Idaho.parquet',
 './data/Google/reviews-estados/review_Oregon.parquet',
 './data/Google/reviews-estados/review_Nevada.parquet',
 './data/Google/reviews-estados/review_West_Virginia.parquet',
 './data/Google/reviews-estados/review-Florida',
 './data/Google/reviews-estados/review_Minnesota.parquet',
 './data/Google/reviews-estados/review-Wyoming',
 './data/Google/reviews-estados/review_Michigan.parquet',
 './data/Google/reviews-estados/review-Georgia',
 './data/Google/reviews-estados/review_Texas.parquet',
 './data/Google/reviews-estados/review-Mississippi',
 './data/Google/reviews-estados/review-Utah',
 './data/Google/reviews-estados/review-Louisiana',
 './data/Google/reviews-estados/review-Massachusetts',
 './data/Google/reviews-estados/review_Vermont.parquet',
 './data/Google/reviews

In [6]:
"""USE ESTE: Este block une todos los JSON en de un dir, convierte la union
   a parquet, y almacena el parquet en ruta del dir
"""
## Convertir archivos JSON, en todo directorio dentro de /reviews-estados/, a parquet

# Iterar por cada directorio dentro de /reviews-estados/
for path_dir in review_dirs:

    if 'parquet' in path_dir.split('.'):
            continue
    
    # Crear lista de JSON dentro de 'path_dir'
    path_jsons = glob(f'{path_dir}/*.json')
    # Crear dataframe donde se unen los datos extraidos de JSON
    union_df = pd.DataFrame()

    # Iterar por cada JSON dentro de 'path_jsons' y almacenar su data en 'union_df'
    for f_json in path_jsons:
        # Leer JSON
        df = pd.read_json(f_json, dtype={'user_id':str}, lines=True)  # fijando 'user_id' a tipo str
        # Unir a union_df
        union_df = pd.concat([union_df, df], ignore_index=True)

    # Almacenar 'union_df' como parquet, en el path './data/Google/reviews-estados/'
    path_almacen = f'{google_dir}/reviews-estados'
    nombre_archivo = f'{path_dir.split("/")[-1].replace("-","_")}'
    union_df.to_parquet(f'{path_almacen}/{nombre_archivo}.parquet')

KeyboardInterrupt: 

In [ ]:
"""NO USAR: Este block convierte JSON a parquet, y almacena el parquet
   resultante en la misma ruta del JSON
"""
"""
## Convertir archivos JSON, en todo directorio dentro de /reviews-estados/, a parquet

# Iterar por cada directorio dentro de /reviews-estados/
for path_dir in review_dirs:
    # Craer lista de JSON dentro de 'path_dir'
    path_jsons = glob(f'{path_dir}/*.json')
    # Iterar por cada JSON dentro de 'path_jsons'
    for f_json in path_jsons:
        # Leer JSON
        df = pd.read_json(f_json, lines=True)
        # Almacenar en formato parquet, en el mismo directorio,
        # cambiando el nombre del archivo (de .json a .parquet)
        df.to_parquet(f_json.replace('.json', '.parquet'))  # se cambia sufijo '.json' a '.parquet'
"""

---
## Tranformacion a CSV, para visualizar

### Yelp

Nota: para dataset Yelp, falta `review.json`

In [ ]:
##  PENDIENTE   !!

#review_path = './data/Yelp/review.json'

In [ ]:
business_path = './data/Yelp/business.parquet'
# Leer parquet
df = pd.read_parquet(business_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{business_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
checkin_path = './data/Yelp/checkin.parquet'
# Leer parquet
df = pd.read_parquet(checkin_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{checkin_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
tip_path = './data/Yelp/tip.parquet'
# Leer parquet
df = pd.read_parquet(tip_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{tip_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
user_path = './data/Yelp/user.parquet'
# Leer parquet
df = pd.read_parquet(user_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{user_path.split("/")[-1].replace(".parquet","")}.csv')

### Google

In [3]:
google_dir = './data/Google'

In [ ]:
# Leer parquet
df = pd.read_parquet(f'{google_dir}/metadata-sitios/1.parquet')
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/google_metadata.csv')

In [4]:
# Leer parquet
df = pd.read_parquet(f'{google_dir}/reviews-estados/review_District_of_Columbia.parquet')
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/google_reviews.csv')